In [ ]:
! pip install pyautogen

In [348]:
import autogen
import os

api_key = os.environ['OPENAI_API_KEY']

config_list = [
    {
        "model": "gpt-4",
        "api_key": api_key
    },
]

In [2]:
SYSTEM_MESSAGE_PRODUCT_MANAGER = """
You are a product manager which works on streaming applications (using Kafka) and all the applications are using python FastStream framework.
You are in a chat with the client and team leader to discuss the requirements for the new application that needs to be implemented.
Go through the given application description, and check if everything what you think is necessary is defined.

Your response should always be ONLY ONE sentence: 
If you receive a question, answer it if you know the answer or respond with "I'm not sure, please check with the user."
If you have a question for the user or team leader please ask it. 
Response with "Everything is clear" if you understand what you need to implement.
"""


SYSTEM_MESSAGE_TEAM_LEADER = """
As the team leader responsible for streaming applications using Kafka and FastStream,
you are in a chat with the client and product manager to discuss the requirements for the new application that needs to be implemented.
Go through the given application description, and check if everything what you think is necessary is defined.

Your response should always be ONLY ONE sentence: 
If you receive a question, answer it if you know the answer or respond with "I'm not sure, please check with the user."
If you have a question for the user or product manager please ask it. 
Response with "Everything is clear" if you understand what you need to implement.
"""


# Construct Agents

In [3]:
llm_config = {"config_list": config_list, "seed": 42}
user_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message="A human admin.",
   code_execution_config={
       "last_n_messages": 2, 
       "work_dir": "groupchat",
       "use_docker": False,  # set to True or image name like "python:3" to use docker
   },
   human_input_mode="ALWAYS"
)
team_leader = autogen.AssistantAgent(
    name="Team_leader",
    system_message=SYSTEM_MESSAGE_TEAM_LEADER,
    llm_config=llm_config,
)

product_manager = autogen.AssistantAgent(
    name="Product_Manager",
    system_message=SYSTEM_MESSAGE_PRODUCT_MANAGER,
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, product_manager, team_leader], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

# Start Chat

In [4]:
description = """
Create a FastStream application which will retrieve the current cryptocurrency price
and publish it to new_crypto_price topic. 

The application should retrieve the data every 2 seconds.

A message which will be produced is JSON with the two attributes:
- price: non-negative float (current price of cryptocurrency in USD)
- crypto_currency: string (the cryptocurrency e.g. BTC, ETH...)


The response of this GET request is a JSON and you can get
information about the crypto_currency in:
    response['data']['base']

and the information about the price in:
    response['data']['amount']

Use utf-8 encoded crypto_currency attribute as a partition key when publishing
the message to new_crypto_price topic.


Is everything clear with the above description?
"""

user_proxy.initiate_chat(manager, message=description)
# type exit to terminate the chat

User_proxy (to chat_manager):


Create a FastStream application which will retrieve the current cryptocurrency price
and publish it to new_crypto_price topic. 

The application should retrieve the data every 2 seconds.

A message which will be produced is JSON with the two attributes:
- price: non-negative float (current price of cryptocurrency in USD)
- crypto_currency: string (the cryptocurrency e.g. BTC, ETH...)


The response of this GET request is a JSON and you can get
information about the crypto_currency in:
    response['data']['base']

and the information about the price in:
    response['data']['amount']

Use utf-8 encoded crypto_currency attribute as a partition key when publishing
the message to new_crypto_price topic.


Is everything clear with the above description?


--------------------------------------------------------------------------------
Product_Manager (to chat_manager):

Which API should we use to retrieve the cryptocurrency price data?

---------------------

In [54]:
groupchat_messages_string = ""
for message in groupchat.messages:
    if "Everything is clear" in message["content"]:
        continue
        
    message["name"] = message["name"].replace("User_proxy", "User")
    message_string = f'{message["name"]}:\n{message["content"]}\n'
    print(message_string)
    groupchat_messages_string += message_string



User:

Create a FastStream application which will retrieve the current cryptocurrency price
and publish it to new_crypto_price topic. 

The application should retrieve the data every 2 seconds.

A message which will be produced is JSON with the two attributes:
- price: non-negative float (current price of cryptocurrency in USD)
- crypto_currency: string (the cryptocurrency e.g. BTC, ETH...)


The response of this GET request is a JSON and you can get
information about the crypto_currency in:
    response['data']['base']

and the information about the price in:
    response['data']['amount']

Use utf-8 encoded crypto_currency attribute as a partition key when publishing
the message to new_crypto_price topic.


Is everything clear with the above description?


Product_Manager:
Which API should we use to retrieve the cryptocurrency price data?

User:
api.coinbase.com/v2/prices

Team_leader:
Are there any specific cryptocurrencies we need to focus on, or should we fetch data for all?

User

## Skeleton generation: user_proxy <--> coder

In [83]:
SYSTEM_MESSAGE_SKELETON_CODER = """
You are an expert Python developer, tasked to generate executable Python code as a part of your work with the FastStream framework. 

You are to abide by the following guidelines:

1. You must never enclose the generated Python code with ``` python. It is mandatory that the output is a valid and executable Python code. Please ensure this rule is never broken.

2. Some prompts might require you to generate code that contains async functions. For example:

async def app_setup(context: ContextRepo):
    raise NotImplementedError()

In such cases, it is necessary to add the "import asyncio" statement at the top of the code. 

3. Whenever you are creating a message class while generating Faststream skeleton and the application code, make sure the message class is a derived class of BaseModel from pydantic.

        Example of a Valid message class:
            class Pet(BaseModel):
                pet_id: NonNegativeInt = Field(..., examples=[1], description="Int data example")
                species: str = Field(..., examples=["dog"], description="Pet example")

        Example of a invalid message class:
            class Pet:
                def __init__(self, pet_id: int, species: str):
                    self.pet_id = pet_id
                    self.species = species
                    
4. When generating a lists of external dependencies from both the ==== APP CODE ==== and ==== TEST CODE ==== sections, include only external libraries and not internal Python libraries like json, time, asyncio, etc., in the ==== APP REQUIREMENT ==== or ==== TEST REQUIREMENT ==== sections. Additionally do not include pytest in the ==== TEST REQUIREMENT ====

You will encounter sections marked as:

==== APP DESCRIPTION: ====

These sections contain the description of the FastStream app you need to implement. Treat everything below this line, until the end of the prompt, as the description to follow for the app implementation.


Reply "TERMINATE" in the end when everything is done.
"""

In [85]:
prompt_with_similar_examples = """

Generate skeleton code for FastStream applications based on provided app descriptions in the "==== USER APP DESCRIPTION ====" section, adhering to these guidelines:

    - Avoid implementing business logic of ANY function. Instead, write "raise NotImplementedError()" and create Google-style docstrings to describe their intended functionality when handling received or produced messages. In each docstring, include a clear instruction to "log the consumed message using logger.info" for subscriber functions.

    - Ensure the generated code aligns with the specific app description requirements.

    - Provide a clear and organized starting point for developers.

    - Your response must contain only valid Python code, saveable as a .py script; no additional text is allowed.
    
    - DO NOT enclose the response within back-ticks. Meaning NEVER ADD ```python to your response.


The goal is to offer developers a structured starting point that matches the example app descriptions, aiding in FastStream application development. Follow the example patterns provided for reference.




==== EXAMPLE APP DESCRIPTION ====

Create a FastStream application with the localhost broker.
Consume from the 'new_pet' topic, which includes JSON encoded object with attributes: pet_id, species, and age.
Whenever a new pet is added, send the new pet's information to the 'notify_adopters' topic.



==== YOUR RESPONSE ====


from pydantic import BaseModel, Field, NonNegativeInt

from faststream import FastStream, Logger
from faststream.kafka import KafkaBroker


class Pet(BaseModel):
    pet_id: NonNegativeInt = Field(..., examples=[1], description="Int data example")
    species: str = Field(..., examples=["dog"], description="Pet example")
    age: NonNegativeInt = Field(..., examples=[1], description="Int data example")


broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


@broker.publisher("notify_adopters")
@broker.subscriber("new_pet")
async def on_new_pet(msg: Pet, logger: Logger) -> Pet:
    \"""
    Processes a message from the 'new_pet' topic and send the new pet's information to the 'notify_adopters' topic.

    Instructions:
    1. Consume a message from 'new_pet' topic.
    2. Send the new pet's information to the 'notify_adopters' topic.

    \"""
    raise NotImplementedError()


==== EXAMPLE APP DESCRIPTION ====

Develop a FastStream application which will fetch weather information from the web until the app shuts down.

You can get the weather information by sending a GET request to "https://api.open-meteo.com/v1/forecast?current_weather=true"
At the end of url you should add additional 'latitude' and 'longitude' parameters which are type float.
Here is url example when you want to fetch information for latitude=52.3 and longitude=13.2:
    "https://api.open-meteo.com/v1/forecast?current_weather=true&latitude=52.3&longitude=13.2"

from the response we want to get info about the temperature (float), windspeed (float) and time (string) and you can find them in:
    response["current_weather"]["temperature"], response["current_weather"]["windspeed"], and response["current_weather"]["time"]

We need to fetch this data every 5 seconds and publish it at 'weather' topic.
For each message you are publishing we must use a key which will be constructed as:
    string value of latitude + '_' + string value of longitude

Message that we will publish needs to have following parameters:
    - latitude (type float)
    - longitude (type float)
    - temperature (type float)
    - windspeed (type float)
    - time (type string)

We need this process for the following latitude and longitude combinations:
    - latitude=13 and longitude=17
    - latitude=50 and longitude=13
    - latitude=44 and longitude=45
    - latitude=24 and longitude=70



==== YOUR RESPONSE ====


from pydantic import BaseModel, Field, NonNegativeFloat

from faststream import ContextRepo, FastStream, Logger
from faststream.kafka import KafkaBroker

broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


class Weather(BaseModel):
    latitude: NonNegativeFloat = Field(
        ...,
        examples=[22.5],
        description="Latitude measures the distance north or south of the equator.",
    )
    longitude: NonNegativeFloat = Field(
        ...,
        examples=[55],
        description="Longitude measures distance east or west of the prime meridian.",
    )
    temperature: float = Field(
        ..., examples=[20], description="Temperature in Celsius degrees"
    )
    windspeed: NonNegativeFloat = Field(
        ..., examples=[20], description="Wind speed in kilometers per hour"
    )
    time: str = Field(
        ..., examples=["2023-09-13T07:00"], description="The time of the day"
    )


@app.on_startup
async def app_setup(context: ContextRepo):
    \"""
    Set all necessary global variables inside ContextRepo object:
        Set app_is_running to True - we will use this variable as running loop condition
    \"""
    raise NotImplementedError()


@app.on_shutdown
async def shutdown(context: ContextRepo):
    \"""
    Set all necessary global variables inside ContextRepo object:
        Set app_is_running to False

    Get all executed tasks from context and wait them to finish
    \"""
    raise NotImplementedError()


async def fetch_and_publish_weather(
    latitude: float,
    longitude: float,
    logger: Logger,
    context: ContextRepo,
    time_inverval: int = 5,
) -> None:
    \"""
    While app_is_running variable inside context is True, repeat the following process:
        get the weather information by sending a GET request to "https://api.open-meteo.com/v1/forecast?current_weather=true"
        At the end of url you should add additional 'latitude' and 'longitude' parameters which are type float.
        Here is url example when you want to fetch information for latitude=52.3 and longitude=13.2:
            "https://api.open-meteo.com/v1/forecast?current_weather=true&latitude=52.3&longitude=13.2"

        from the response we want to get info about the temperature (float), windspeed (float) and time (string) and you can find them in:
            response["current_weather"]["temperature"], response["current_weather"]["windspeed"], and response["current_weather"]["time"]

        We need to fetch this data, construct the Weather object and publish it at 'weather' topic.
        For each message you are publishing we must use a key which will be constructed as:
            string value of latitude + '_' + string value of longitude

        asynchronous sleep for time_interval
    \"""
    raise NotImplementedError()


@app.after_startup
async def publish_weather(logger: Logger, context: ContextRepo):
    \"""
    Create asynchronous tasks for executing fetch_and_publish_weather function.
    Run this process for the following latitude and longitude combinations:
        - latitude=13 and longitude=17
        - latitude=50 and longitude=13
        - latitude=44 and longitude=45
        - latitude=24 and longitude=70
    Put all executed tasks to list and set it as global variable in context (It is needed so we can wait for this tasks at app shutdown)
    \"""
    raise NotImplementedError()


==== EXAMPLE APP DESCRIPTION ====

Develop a FastStream application using localhost kafka broker.
The app should consume messages from the input_data topic.
The input message is a JSON encoded object including two attributes:
    - x: float
    - y: float
    - time: datetime

input_data topic should use partition key.
While consuming the message, increment x and y attributes by 1 and publish that message to the output_data topic.
The same partition key should be used in the input_data and output_data topic.



==== YOUR RESPONSE ====


from datetime import datetime

from pydantic import BaseModel, Field

from faststream import Context, FastStream, Logger
from faststream.kafka import KafkaBroker


class Point(BaseModel):
    x: float = Field(
        ..., examples=[0.5], description="The X Coordinate in the coordinate system"
    )
    y: float = Field(
        ..., examples=[0.5], description="The Y Coordinate in the coordinate system"
    )
    time: datetime = Field(
        ...,
        examples=["2020-04-23 10:20:30.400000"],
        description="The timestamp of the record",
    )


broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


to_output_data = broker.publisher("output_data")


@broker.subscriber("input_data")
async def on_input_data(
    msg: Point, logger: Logger, key: bytes = Context("message.raw_message.key")
) -> None:
    \"""
    Processes a message from the 'input_data' topic.
    Increment msg x and y attributes with 1 and publish that message to the output_data topic.
    The same partition key should be used in the input_data and output_data topic.

    Instructions:
    1. Consume a message from 'input_data' topic.
    2. Create a new message object (do not directly modify the original).
    3. Increment msg x and y attributes with 1.
    4. Publish that message to the output_data topic (The same partition key should be used in the input_data and output_data topic).
    \"""
    raise NotImplementedError()
    
    
==== USER APP DESCRIPTION: ====

"""

In [86]:
# messages = [{'content': prompt_with_similar_examples, 'role': 'user', 'name': 'Coder'}]
# messages_with_examples_and_description = messages + groupchat.messages
# messages_with_examples_and_description

In [100]:
prompt = prompt_with_similar_examples + groupchat_messages_string + "Please create skeleton application for the given description."
prompt

'\n\nGenerate skeleton code for FastStream applications based on provided app descriptions in the "==== USER APP DESCRIPTION ====" section, adhering to these guidelines:\n\n    - Avoid implementing business logic of ANY function. Instead, write "raise NotImplementedError()" and create Google-style docstrings to describe their intended functionality when handling received or produced messages. In each docstring, include a clear instruction to "log the consumed message using logger.info" for subscriber functions.\n\n    - Ensure the generated code aligns with the specific app description requirements.\n\n    - Provide a clear and organized starting point for developers.\n\n    - Your response must contain only valid Python code, saveable as a .py script; no additional text is allowed.\n    \n    - DO NOT enclose the response within back-ticks. Meaning NEVER ADD ```python to your response.\n\n\nThe goal is to offer developers a structured starting point that matches the example app descri

In [194]:
import random

config_list = [
    {
        "model": "gpt-3.5-turbo-16k",
        "api_key": api_key
    },
]

llm_config = {
    "config_list": config_list,
    "seed": random.randint(100, 900),
    "temperature": 0.7,  # temperature for sampling
}  # configuration for autogen's enhanced inference API which is compatible with OpenAI API


USER_PROXY_SYSTEM_MESSAGE = """
You are a QA engineer with a simple task that you need to check.
Check that all functions and methods which the Coder sends to you are NOT implemented, each function should just raise NotImplementedError().
If some function or method is implemented, reply to the the Coder to delete the implementation and just to replace it with "raise NotImplementedError()"
You should never write any code, just anlyze, execute the code and write instructions to the coder what needs to be fixed.

If there are any external dependencies (such as FastStream) please install them.

If the code looks good to you, save it in the application_skeleton.py and execute it.
Your job can not be done until you execute application_skeleton.py!
If there are not any errors with application_skeleton.py reply with "# TERMINATE".
otherwise, reply with the instructions what needs to be fixed. NEVER reply with any code, just instructions!

Reply "# TERMINATE" in the end when everything is done.
"""

code_execution_config={
       "work_dir": "groupchat_skeleton",
       "use_docker": False,  # set to True or image name like "python:3" to use docker
}

user_skeleton_proxy = autogen.UserProxyAgent(
    name="User_skeleton_proxy",
    system_message=USER_PROXY_SYSTEM_MESSAGE,
    llm_config=llm_config,
    code_execution_config=code_execution_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
)

CODER_SYSTEM_MESSAGE = """
Your job is to write application_skeleton.py for the given application description.
The application must use FastStream framework. 
For each function and method write the docstring and raise NotImplementedError() i.e. do NOT implement any function.

If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply "TERMINATE" in the end when everything is done.
"""

coder = autogen.AssistantAgent(
    name="Coder",
    system_message=CODER_SYSTEM_MESSAGE,
    llm_config=llm_config,    
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
)



In [195]:
user_skeleton_proxy.initiate_chat(
    coder,
    message="# filename: application_skeleton.py; " + prompt + "\n# filename: application_skeleton.py; ",
)

User_skeleton_proxy (to Coder):

# filename: application_skeleton.py; 

Generate skeleton code for FastStream applications based on provided app descriptions in the "==== USER APP DESCRIPTION ====" section, adhering to these guidelines:

    - Avoid implementing business logic of ANY function. Instead, write "raise NotImplementedError()" and create Google-style docstrings to describe their intended functionality when handling received or produced messages. In each docstring, include a clear instruction to "log the consumed message using logger.info" for subscriber functions.

    - Ensure the generated code aligns with the specific app description requirements.

    - Provide a clear and organized starting point for developers.

    - Your response must contain only valid Python code, saveable as a .py script; no additional text is allowed.
    
    - DO NOT enclose the response within back-ticks. Meaning NEVER ADD ```python to your response.


The goal is to offer developers a structu


--------------------------------------------------------------------------------
Coder (to User_skeleton_proxy):

from pydantic import BaseModel, Field, NonNegativeFloat
from datetime import datetime
from faststream import Context, FastStream, Logger
from faststream.kafka import KafkaBroker

class CryptoPrice(BaseModel):
    price: NonNegativeFloat = Field(..., examples=[10000.0], description="The current price of the cryptocurrency in USD")
    crypto_currency: str = Field(..., examples=["BTC", "ETH"], description="The cryptocurrency symbol")

broker = KafkaBroker("localhost:9092")
app = FastStream(broker)

to_new_crypto_price = broker.publisher("new_crypto_price")

@app.on_startup
async def app_setup():
    """
    Set all necessary global variables inside Context object:
        Set app_is_running to True - we will use this variable as running loop condition
    """
    raise NotImplementedError()

@app.on_shutdown
async def shutdown():
    """
    Set all necessary global variable

## Simple example with saving the generated code to .py file

In [141]:
config_list = [
    {
        "model": "gpt-4",
        "api_key": api_key
    },
]

conversations = {}
autogen.ChatCompletion.start_logging(conversations)

# create an AssistantAgent named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "seed": 22,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0,  # temperature for sampling
    },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
)

DEFAULT_SYSTEM_MESSAGE = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.

Please save the created code into a python file.
Reply "TERMINATE" in the end when everything is done.
"""
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,  # set to True or image name like "python:3" to use docker
    },
)
# the assistant receives a message from the user_proxy, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""# filename: saved_code.py; What date is today? Compare the year-to-date gain for META and TESLA.""",
)

user_proxy (to assistant):

# filename: saved_code.py; What date is today? Compare the year-to-date gain for META and TESLA.

--------------------------------------------------------------------------------
assistant (to user_proxy):

To solve this task, we will need to do the following:

1. Get the current date using Python's datetime module.
2. Fetch the stock prices for META (Facebook) and TESLA for the current year and the previous year. We can use the yfinance module in Python to fetch this data.
3. Calculate the year-to-date gain for both stocks and compare them.

Let's start with the first step. Here is the Python code to get the current date:

```python
# filename: saved_code.py

import datetime

# Get the current date
current_date = datetime.date.today()

print("Today's date is:", current_date)
```

Please run this code to get the current date. After that, we will proceed with fetching the stock prices.

-------------------------------------------------------------------------

Great! The code has successfully fetched the stock prices for META and TESLA and calculated the year-to-date gain for both. 

As per the output:

- META's year-to-date gain is 155.22%
- TESLA's year-to-date gain is 140.21%

This means that so far in this year, META's stock price has increased by 155.22% from the price at the start of the year, while TESLA's stock price has increased by 140.21% from the price at the start of the year. 

Therefore, comparing the two, META has had a higher year-to-date gain than TESLA.

TERMINATE

--------------------------------------------------------------------------------


## Skeleton generation groupchat with functions: user_proxy, coder, QA

In [285]:
import random

config_list = [
#     {
#         "model": "gpt-4",
#         "api_key": api_key
#     },
    {
        "model": "gpt-3.5-turbo-16k",
        "api_key": api_key
    },
]

llm_config_qa = {
    "functions": [
        {
            "name": "python",
            "description": "run cell in ipython and return the execution result.",
            "parameters": {
                "type": "object",
                "properties": {
                    "cell": {
                        "type": "string",
                        "description": "Valid Python cell to execute.",
                    }
                },
                "required": ["cell"],
            },
        },
    ],
    "config_list": config_list,
    "seed": random.randint(100, 900),
    "temperature": 0.7,  # temperature for sampling
}  

llm_config = {
    "config_list": config_list,
    "seed": random.randint(100, 900),
    "temperature": 0.7,  # temperature for sampling
} 

# You are a proxy between the Coder and the Quality Assurance Engineer. All messages received from the Coder forward to the Quality Assurance Engineer.
USER_PROXY_SYSTEM_MESSAGE = """


Each time you receive a message from the Coder, follow these steps: 
- If the code looks good to you, save it in the application_skeleton.py and execute it.
- Your job can not be done until you execute application_skeleton.py!
- If there are not any errors once you execute the application_skeleton.py reply with "# TERMINATE".
- But you can't reply with TERMINATE if you did not execute the received code!!!
- If there are some problems with the code, reply with the instructions what needs to be fixed. NEVER reply with any code, just instructions!
- When something needs to be fixed, send it to the Coder

If the code execution is successful, reply with "TERMINATE".
"""
# Reply "# TERMINATE" in the end when everything is done.

code_execution_config={
       "work_dir": "groupchat_skeleton",
       "use_docker": False,  # set to True or image name like "python:3" to use docker
}

user_skeleton_proxy = autogen.UserProxyAgent(
    name="User_skeleton_proxy",
    system_message=USER_PROXY_SYSTEM_MESSAGE,
    llm_config=llm_config,
    code_execution_config=code_execution_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if (x and "TERMINATE" in x.get("content", "")) else False,
)

# CODER_SYSTEM_MESSAGE = """
# Your job is to write application_skeleton.py for the given application description.
# The application must use FastStream framework. 
# For each function and method write the docstring and raise NotImplementedError() i.e. do NOT implement any function.
# The code you create MUST be inside the ```python block!

# If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
# If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
# When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
# """

CODER_SYSTEM_MESSAGE = """
Generate skeleton code for FastStream applications based on provided app descriptions in the "==== USER APP DESCRIPTION ====" section, adhering to these guidelines:

    - Avoid implementing business logic of ANY function. Instead, write "raise NotImplementedError()" and create Google-style docstrings to describe their intended functionality when handling received or produced messages. In each docstring, include a clear instruction to "log the consumed message using logger.info" for subscriber functions.

    - Ensure the generated code aligns with the specific app description requirements.

    - Provide a clear and organized starting point for developers.

    - Your response must contain only valid Python code within back-ticks
    
    - ALWAYS enclose the response within back-ticks. Meaning ALWAYS ADD ```python to your response.
    
    - NEVER add main function and "if __name__ == "__main__":" block


The goal is to offer developers a structured starting point that matches the example app descriptions, aiding in FastStream application development. Follow the example patterns provided for reference.
"""

coder = autogen.AssistantAgent(
    name="Coder",
    system_message=CODER_SYSTEM_MESSAGE,
    llm_config=llm_config,    
    is_termination_msg=lambda x: True if (x and "TERMINATE" in x.get("content", "")) else False,
)

QA_SYSTEM_MESSAGE = """
You are a QA engineer with 3 simple task that you need to check.
1.
Check that all functions and methods which the Coder sends to you are NOT implemented, each function should just raise NotImplementedError().
If some function or method is implemented, reply to the the Coder to delete the implementation and just to replace it with "raise NotImplementedError()"
2.
Check that the IS NOT main function and "if __name__ == "__main__":" block.
If the main block is implemented, tell the Coder to delete it
3. The code is written inside the  within back-ticks - ```python

Do NOT write any functions to test that, just take a look at the code and write instructions to the Coder if some function is implemented.
You should never write any NEW code.
If the code looks ok, forward it to User_skeleton_proxy.
"""

quality_assurance_engineer = autogen.AssistantAgent(
    name="QA_Engineer",
    system_message=QA_SYSTEM_MESSAGE,
    llm_config=llm_config_qa,    
    is_termination_msg=lambda x: True if (x and "TERMINATE" in x.get("content", "")) else False,
)


In [286]:
# define functions according to the function desription
from IPython import get_ipython

def exec_python(cell):
    ipython = get_ipython()
    result = ipython.run_cell(cell)
    log = str(result.result)
    if result.error_before_exec is not None:
        log += f"\n{result.error_before_exec}"
    if result.error_in_exec is not None:
        log += f"\n{result.error_in_exec}"
    return log


# register the functions
user_skeleton_proxy.register_function(
    function_map={
        "python": exec_python,
    }
)

# user_skeleton_proxy.initiate_chat(
#     coder,
#     message="# filename: application_skeleton.py; " + prompt,
# )

In [287]:
CHAT_MANAGER_SYSTEM_MESSAGE = """
You are a Group chat manager.

Always forward the message from the Coder to the Quality Ensurance Engineer.

If the Quality Ensurance Engineer returns you ***** Suggested function Call: python *****, forward the message to the User_skeleton_proxy to execute it.
Otherwise, forward the message to the Coder.
"""

groupchat_skeleton = autogen.GroupChat(agents=[user_skeleton_proxy, coder, quality_assurance_engineer], messages=[], max_round=6)
manager_skeleton = autogen.GroupChatManager(groupchat=groupchat_skeleton, llm_config=llm_config)
user_skeleton_proxy.initiate_chat(manager_skeleton, message="# filename: application_skeleton.py; " + prompt)

# type exit to terminate the chat

User_skeleton_proxy (to chat_manager):

# filename: application_skeleton.py; 

Generate skeleton code for FastStream applications based on provided app descriptions in the "==== USER APP DESCRIPTION ====" section, adhering to these guidelines:

    - Avoid implementing business logic of ANY function. Instead, write "raise NotImplementedError()" and create Google-style docstrings to describe their intended functionality when handling received or produced messages. In each docstring, include a clear instruction to "log the consumed message using logger.info" for subscriber functions.

    - Ensure the generated code aligns with the specific app description requirements.

    - Provide a clear and organized starting point for developers.

    - Your response must contain only valid Python code, saveable as a .py script; no additional text is allowed.
    
    - DO NOT enclose the response within back-ticks. Meaning NEVER ADD ```python to your response.


The goal is to offer developers a 


--------------------------------------------------------------------------------
Coder (to chat_manager):

from pydantic import BaseModel, Field, NonNegativeFloat
from datetime import datetime
from faststream import Context, FastStream, Logger
from faststream.kafka import KafkaBroker
import requests
import asyncio

class CryptoPrice(BaseModel):
    price: NonNegativeFloat = Field(..., examples=[10000.0], description="Current price of cryptocurrency in USD")
    crypto_currency: str = Field(..., examples=["BTC"], description="The cryptocurrency e.g. BTC, ETH...")
    
broker = KafkaBroker("localhost:9092")
app = FastStream(broker)

to_new_crypto_price = broker.publisher("new_crypto_price")

async def fetch_and_publish_crypto_price(logger: Logger, time_interval: int = 2) -> None:
    """
    Fetches the current cryptocurrency price from the API every 2 seconds and publishes it to the 'new_crypto_price' topic.
    The message is a JSON object with attributes price and crypto_currency.

 

TypeError: argument of type 'NoneType' is not iterable

In [291]:
d = {
"cell": "# filename: application_skeleton.py; \n\nGenerate skeleton code for FastStream applications based on provided app descriptions in the \"==== USER APP DESCRIPTION ====\" section, adhering to these guidelines:\n\n    - Avoid implementing business logic of ANY function. Instead, write \"raise NotImplementedError()\" and create Google-style docstrings to describe their intended functionality when handling received or produced messages. In each docstring, include a clear instruction to \"log the consumed message using logger.info\" for subscriber functions.\n\n    - Ensure the generated code aligns with the specific app description requirements.\n\n    - Provide a clear and organized starting point for developers.\n\n    - Your response must contain only valid Python code, saveable as a .py script; no additional text is allowed.\n    \n    - DO NOT enclose the response within back-ticks. Meaning NEVER ADD ```python to your response.\n\n\nThe goal is to offer developers a structured starting point that matches the example app descriptions, aiding in FastStream application development. Follow the example patterns provided for reference.\n\n\n\n==== EXAMPLE APP DESCRIPTION ====\n\nCreate a FastStream application with the localhost broker.\nConsume from the 'new_pet' topic, which includes JSON encoded object with attributes: pet_id, species, and age.\nWhenever a new pet is added, send the new pet's information to the 'notify_adopters' topic.\n\n\n\n==== YOUR RESPONSE ====\n\n\nfrom pydantic import BaseModel, Field, NonNegativeInt\n\nfrom faststream import FastStream, Logger\nfrom faststream.kafka import KafkaBroker\n\n\nclass Pet(BaseModel):\n    pet_id: NonNegativeInt = Field(..., examples=[1], description=\"Int data example\")\n    species: str = Field(..., examples=[\"dog\"], description=\"Pet example\")\n    age: NonNegativeInt = Field(..., examples=[1], description=\"Int data example\")\n\n\nbroker = KafkaBroker(\"localhost:9092\")\napp = FastStream(broker)\n\n\n@broker.publisher(\"notify_adopters\")\n@broker.subscriber(\"new_pet\")\nasync def on_new_pet(msg: Pet, logger: Logger) -> Pet:\n    \"\"\"\n    Processes a message from the 'new_pet' topic and send the new pet's information to the 'notify_adopters' topic.\n\n    Instructions:\n    1. Consume a message from 'new_pet' topic.\n    2. Send the new pet's information to the 'notify_adopters' topic.\n\n    \"\"\"\n    raise NotImplementedError()\n\n\n",
}
print(d and "TERMINATE" in d.get("content", ""))

False


In [349]:
import random

config_list = [
#     {
#         "model": "gpt-4",
#         "api_key": api_key
#     },
    {
        "model": "gpt-3.5-turbo-16k",
        "api_key": api_key
    },
]

llm_config_qa = {
    "functions": [
        {
            "name": "python",
            "description": "run cell in ipython and return the execution result.",
            "parameters": {
                "type": "object",
                "properties": {
                    "cell": {
                        "type": "string",
                        "description": "Valid Python cell to execute.",
                    }
                },
                "required": ["cell"],
            },
        },
    ],
    "config_list": config_list,
    "seed": random.randint(100, 900),
    "temperature": 0.7,  # temperature for sampling
}  

llm_config = {
    "config_list": config_list,
    "seed": random.randint(100, 900),
    "temperature": 0.7,  # temperature for sampling
} 

# You are a proxy between the Coder and the Quality Assurance Engineer. All messages received from the Coder forward to the Quality Assurance Engineer.
USER_PROXY_SYSTEM_MESSAGE = """


Each time you receive a message from the Coder, follow these steps: 
- If the code looks good to you, save it in the application_skeleton.py and execute it.
- Your job can not be done until you execute application_skeleton.py!
- If there are not any errors once you execute the application_skeleton.py reply with "# TERMINATE".
- But you can't reply with TERMINATE if you did not execute the received code!!!
- If there are some problems with the code, reply with the instructions what needs to be fixed. NEVER reply with any code, just instructions!
- When something needs to be fixed, send it to the Coder

The code execution should succesfully pass - "exitcode: 0 (execution succeeded)"
If the code execution is successful, reply with "TERMINATE".
"""
# Reply "# TERMINATE" in the end when everything is done.

code_execution_config={
       "work_dir": "groupchat_skeleton",
       "use_docker": False,  # set to True or image name like "python:3" to use docker
}

user_skeleton_proxy = autogen.UserProxyAgent(
    name="User_skeleton_proxy",
    system_message=USER_PROXY_SYSTEM_MESSAGE,
    llm_config=llm_config,
    code_execution_config=code_execution_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if (x.get("content") and "TERMINATE" in x.get("content")) else False,
)

# CODER_SYSTEM_MESSAGE = """
# Your job is to write application_skeleton.py for the given application description.
# The application must use FastStream framework. 
# For each function and method write the docstring and raise NotImplementedError() i.e. do NOT implement any function.
# The code you create MUST be inside the ```python block!

# If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
# If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
# When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
# """

CODER_SYSTEM_MESSAGE = """
Generate skeleton code for the code, adhering to these guidelines:

    - Avoid implementing business logic of ANY function. Instead, write "raise NotImplementedError()" and create Google-style docstrings to describe their intended functionality when handling received or produced messages. In each docstring, include a clear instruction to "log the consumed message using logger.info" for subscriber functions.

    - Ensure the generated code aligns with the specific app description requirements.

    - Provide a clear and organized starting point for developers.

    - Your response must contain only valid Python code within back-ticks
    
    - ALWAYS enclose the response within back-ticks. Meaning ALWAYS ADD ```python to your response.
    
    - NEVER add main function and "if __name__ == "__main__":" block


The goal is to offer developers a structured starting point that matches the example app descriptions.
Follow the example patterns provided for reference.
"""

coder = autogen.AssistantAgent(
    name="Coder",
    system_message=CODER_SYSTEM_MESSAGE,
    llm_config=llm_config,    
    is_termination_msg=lambda x: True if (x.get("content") and "TERMINATE" in x.get("content")) else False,
)

JUNIOR_SYSTEM_MESSAGE = """
You are a junior developer with 3 simple task that you need to check.
1.
Check that all functions and methods which the Coder sends to you are NOT implemented, each function should just raise NotImplementedError().
If some function or method is implemented, reply to the the Coder to delete the implementation and just to replace it with "raise NotImplementedError()"
2.
Check that the IS NOT main function and "if __name__ == "__main__":" block.
If the main block is implemented, tell the Coder to delete it
3. The code is written inside the  within back-ticks - ```python

Do NOT write any functions to test that, just take a look at the code and write instructions to the Coder if some function is implemented.
You should never write any NEW code.

If the code looks ok, send the Suggested function Call: python, to User_skeleton_proxy so he can execute it.
"""

quality_assurance_engineer = autogen.AssistantAgent(
    name="Junior_developer",
    system_message=JUNIOR_SYSTEM_MESSAGE,
    llm_config=llm_config_qa,    
    is_termination_msg=lambda x: True if (x.get("content") and "TERMINATE" in x.get("content")) else False,

)

user_skeleton_proxy.register_function(
    function_map={
        "python": exec_python,
    }
)

CHAT_MANAGER_SYSTEM_MESSAGE = """
You are a Group chat manager.

These are the rules how to select the next speeker:
- The first message must be sent to the Coder.
- ALL messages from the Coder forward to the Junior_developer.
- ALL messages from the Junior_developer forward to the User_skeleton_proxy.
- ALL messages from the User_skeleton_proxy forward to the Coder.
"""

groupchat_skeleton = autogen.GroupChat(agents=[user_skeleton_proxy, coder, quality_assurance_engineer], messages=[], max_round=12)
manager_skeleton = autogen.GroupChatManager(groupchat=groupchat_skeleton, llm_config=llm_config, system_message=CHAT_MANAGER_SYSTEM_MESSAGE)
user_skeleton_proxy.initiate_chat(manager_skeleton, message="# filename: application_skeleton.py; " + "Write FastAPI application skeleton for item creation")

User_skeleton_proxy (to chat_manager):

# filename: application_skeleton.py; Write FastAPI application skeleton for item creation

--------------------------------------------------------------------------------


KeyboardInterrupt: 